In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [5]:
from groq import Groq
from dotenv import load_dotenv
import os
load_dotenv()

True

In [7]:
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt):
    response = groq_client.chat.completions.create(
        model='meta-llama/llama-4-maverick-17b-128e-instruct',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [12]:
rag('how do I run kafka?')

'To run Kafka, it appears there are different methods based on the programming language and specific requirements you\'re dealing with.\n\nFor Java Kafka, you can run a producer or consumer by executing the following command in the project directory:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor Python Kafka, ensure you have the correct environment setup:\n1. Create a virtual environment and install the necessary packages by running:\n   ```bash\n   python -m venv env\n   source env/bin/activate  # or env/Scripts/activate on Windows\n   pip install -r ../requirements.txt\n   ```\n2. Activate the virtual environment every time you need it:\n   ```bash\n   source env/bin/activate  # or env/Scripts/activate on Windows\n   ```\n3. Ensure Docker images are up and running before executing your Python files.\n\nIf you encounter a "Permission denied" error when running `./build.sh`, use:\n```bash\nchmod +x build.sh\n```\ni

In [13]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course after it has started. You will still be eligible to submit the homeworks, but be aware that there will be deadlines for turning in the final projects.'

## RAG with Vector Search

In [14]:
from qdrant_client import QdrantClient, models

c:\Users\admin_ABH\anaconda3\envs\ai-engineering\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\admin_ABH\anaconda3\envs\ai-engineering\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:27: UserWarning: Unsupported Windows version (2022server). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [15]:
qd_client = QdrantClient("http://localhost:6333")

In [16]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [17]:
collection_name = "zoomcamp-faq"

In [18]:
qd_client.delete_collection(collection_name=collection_name)

True

In [19]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [20]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [22]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [23]:
question = 'I just discovered the course. Can I still join it?'

In [24]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [25]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [26]:
rag('how do I run kafka?')

vector_search is used


"To run Kafka, based on the provided context, here are the steps you may need to follow depending on your specific task:\n\n1. **For Java Kafka Producer/Consumer/KStreams/etc**: To run these in the terminal, navigate to your project directory and use the command:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Make sure to replace `<jar_name>` with your actual jar file name. Also, verify that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` is set to the correct server URL in your Java scripts (e.g., `JsonConsumer.java`, `JsonProducer.java`), and that your cluster key and secrets are updated in `src/main/java/org/example/Secrets.java`.\n\n2. **Troubleshooting `kafka.errors.NoBrokersAvailable: NoBrokersAvailable`**: If you encounter this error, it's likely your Kafka broker Docker container isn't working. Check with `docker ps`, then run `docker compose up -d` in the Docker Compose YAML file folder to start all instance